In [68]:
import pandas as pd
import os
import openai
import os
from dotenv import load_dotenv
from openai import OpenAI 
from IPython.display import display, Markdown

In [55]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
model_natural_lang = 'gpt-4o'

In [56]:
biografias_df = pd.read_csv('/Users/elorr/Documents/backstage/dados/biografias.csv')
linkedin_posts_df = pd.read_csv('/Users/elorr/Documents/backstage/dados/noticias.csv')
artigos_df = pd.read_csv('/Users/elorr/Documents/backstage/dados/posts_linkedin.csv')

# Função extração de info

In [57]:
def extrair_informacoes(conteudo, tipo):

    if tipo == 'biogragia':
        completion = client.chat.completions.create(
        model=model_natural_lang,
            messages=[
                {"role": "system", "content": "Você é um assistente que ajuda a extrair informações específicas de biografias."},
                {"role": "user", "content": f"Por favor, extraia os seguintes tópicos da biografia fornecida: {conteudo}."},
                {"role": "user", "content": """
                    - Contexto
                    - Experiências
                    - Trajetória Profissional
                    - Contribuições
                    - Impacto na Comunidade
                    - Competências
                    - Assuntos Dominados
                    - Formas de Agregar para o Leitor
                """}
            ]
        )
    else:
        completion = client.chat.completions.create(
        model=model_natural_lang,
        messages=[
            {"role": "system", "content": "Você é um assistente que ajuda a extrair informações específicas de noticias e artigos de uma personalidade. Por meio do artigo fornecido, eu quero que voce identifique todas as caracteristicas de escrita dessa personalidade para que voce possa reproduzir outros textos sobre outras temáticas como exatamente a maneira que essa personalidade escreveria"},
            {"role": "user", "content": f"Por favor, extraia os seguintes tópicos do artigo fornecido: {conteudo}."},
            {"role": "user", "content": """
                - Escolha de Palavras: identifique as palavras e expressões frequentes e distintivas.
                - Complexidade Lexical: Medir a diversidade e a complexidade do vocabulário utilizado.
                - Estrutura das Frases: Analisar a estrutura gramatical das frases, incluindo o uso de frases curtas e longas.
                - Uso de Pontuação: Observar o uso de vírgulas, pontos, travessões, pontos de exclamação e interrogação.
                - Tom de Voz: Determinar se o tom é assertivo, reflexivo, sarcástico, etc.
                - Formalidade: Avaliar o nível de formalidade do texto.
                - Persuasividade: Analisar técnicas persuasivas, como apelos emocionais e argumentos lógicos.
                - Sentimento: Avaliar a polaridade (positiva, negativa, neutra) e a intensidade emocional do texto.
                - Temas e Tópicos: Identificar os principais temas e tópicos abordados no texto.
                - Figura de Linguagem: Detectar o uso de metáforas, símiles, ironias, hipérboles, etc.
                - Uso de Repetição: Identificar padrões de repetição de palavras e frases para ênfase.
                - Coerência: Avaliar a lógica e a fluidez das ideias apresentadas no texto.
                - Entonação: Estudar a entonação das frases.
                - Ritmo e Pausas: Analisar o ritmo da fala e o uso de pausas.
                - Marcas de Identidade: Detectar elementos que indicam a identidade pessoal ou profissional do autor.
                - Referências Pessoais: Observar menções a experiências pessoais, crenças, e valores.
            """}
        ]
    )
        return completion.choices[0].message.content


# Extrair informações da biografia

In [59]:
resultados = []

for index, row in biografias_df.iterrows():
    conteudo = row['conteudo']
    tipo = row['tipo']
    informacoes = extrair_informacoes(conteudo, tipo)
    resultados.append({'id': row['id'], 'informacoes': informacoes})

resultados_biografias_df = pd.DataFrame(resultados)


# Extrair informações das notícias 

In [71]:
for index, row in artigos_df.iterrows():
    conteudo = row['conteudo']
    tipo = row['tipo']
    informacoes = extrair_informacoes(conteudo, tipo)
    resultados.append({'id': row['id'], 'informacoes': informacoes})

resultados_artigos_df = pd.DataFrame(resultados)
print(resultados_artigos_df[0])

KeyError: 0

# Extrair informações dos posts linkedin 

In [61]:
for index, row in linkedin_posts_df.iterrows():
    conteudo = row['conteudo']
    tipo = row['tipo']
    informacoes = extrair_informacoes(conteudo, tipo)
    resultados.append({'id': row['id'], 'informacoes': informacoes})

resultados_linkedin_df = pd.DataFrame(resultados)


# O prompt ideal segue a seguinte estrutura:

CONTEXTO + TASK + EXEMPLO + PERSONA + FORMATO + TOM DE VOZ 

# GPT Final 

Converter dataframe para texto

In [65]:
def df_para_texto(df):
    return " ".join(df['informacoes'].tolist())

biografia_texto = df_para_texto(resultados_biografias_df)
artigos_texto = df_para_texto(resultados_artigos_df)
linkedin_texto = df_para_texto(resultados_artigos_df)


# Converter exemplos de posts do LinkedIn para texto
linkedin_posts_texto = " ".join(linkedin_posts_df['conteudo'].tolist()[:3])

In [75]:
biografia_texto

'### Análise do Texto de Ricardo Basaglia:\n\n#### Escolha de Palavras:\n- **Palavras e Expressões Frequentes e Distintivas**:\n  - "transformação digital"\n  - "vida das pessoas"\n  - "startup de negócios"\n  - "operações"\n  - "liderança"\n  - "best-seller"\n  - "impactando milhares de pessoas"\n\n#### Complexidade Lexical:\n- **Vocabulário Utilizado**:\n  - O texto utiliza um vocabulário acessível, porém sofisticado, relevante ao contexto corporativo e de liderança.\n  - Termos como "transformação digital," "estratégias," "operações," e "impacto" são indicativos de uma narrativa própria de negócios e gestão.\n  - A complexidade lexical é moderada, com palavras técnicas e específicas da área de administração e tecnologia.\n\n#### Estrutura das Frases:\n- **Estrutura Gramatical das Frases**:\n  - São utilizadas frases mistas: algumas curtas e impactantes, e outras mais longas e detalhadas.\n  - Uso de oração subordinada para explicar contexto, como "Iniciou a carreira na área de tecno

In [79]:
def gpt_final(biografia_texto, artigos_texto, linkedin_texto, NUM_CARACTERES, TEMA , linkedin_posts_texto):
    completion = client.chat.completions.create(
    model=model_natural_lang,
        messages=[
          {"role": "system", "content": f"Você é a melhor agência de marketing de conteúdo estratégico para o Linkedin. Você sabe escrever os textos mais coesos, claros, explicativos, e objetivos para o seu cliente. A fim de gerar textos escritos customizados exatamente do jeito que o cliente se comunica, sobre os assuntos que ele aborda e de uma maneira que cative qualquer público leitor a ler, você deve considerar os seguintes aspectos: 1. Entender quem é o cliente em análise. Por meio da consulta desses dados {biografia_texto}.2. Aprender a escrever com as mesmas características que o cliente escreve. Esses dados estão presentes em {linkedin_texto} e {artigos_texto}. 3. Com base na consulta de todos esses dados, quero que escreva um texto de {NUM_CARACTERES} caracteres sobre {TEMA} em forma de parágrafos. 4. Finalmente, sugira novos assuntos que você acha pertinente esse cliente falar com base em todos os temas e tópicos já abordados.  Segue alguns exemplos de posts no linkedin já feitos: {linkedin_posts_texto}."} # <-- This is the system message that provides context to the model
          # {"role": "user", "content": f"Olá! Quero escrever um texto sobre {TEMA} de acordo com as características do meu cliente de {NUM_CARACTERES} caracteres em forma de parágrafos"}
        ]
    )
    return completion.choices[0].message.content

tema = "A melhor maneira de dar feeback para seus liderados"
num_caracteres = 600
texto_final = gpt_final(biografia_texto, artigos_texto, linkedin_texto, num_caracteres, tema,linkedin_posts_texto)
print(texto_final)

# Exibir o texto formatado usando Markdown
display(Markdown(texto_final))

### Melhor Maneira de Dar Feedback para seus Liderados

A arte de dar feedback é uma ferramenta essencial para líderes que desejam promover crescimento e melhorar performances. Primeiramente, crie um ambiente seguro e de confiança para que seus liderados se sintam à vontade para receber e dar feedback. Seja específico e claro ao abordar comportamentos e resultados, evitando generalizações. Use a técnica "sanduíche", iniciando com um elogio, seguido pelo feedback construtivo e finalizando com uma nota positiva. Encoraje o diálogo, ouvindo ativamente as percepções dos colaboradores. Isso não só mostra respeito, mas também fortalece a relação e facilita a aceitação e implementação do feedback.

### Sugestão de Novos Assuntos

1. **Autoconhecimento e Liderança:** Como o autoconhecimento pode tornar líderes mais eficazes.
2. **Inteligência Emocional no Trabalho:** Desenvolvendo inteligência emocional para melhorar o ambiente corporativo.
3. **Gestão de Equipes Remotas:** Estrategias e ferra

### Melhor Maneira de Dar Feedback para seus Liderados

A arte de dar feedback é uma ferramenta essencial para líderes que desejam promover crescimento e melhorar performances. Primeiramente, crie um ambiente seguro e de confiança para que seus liderados se sintam à vontade para receber e dar feedback. Seja específico e claro ao abordar comportamentos e resultados, evitando generalizações. Use a técnica "sanduíche", iniciando com um elogio, seguido pelo feedback construtivo e finalizando com uma nota positiva. Encoraje o diálogo, ouvindo ativamente as percepções dos colaboradores. Isso não só mostra respeito, mas também fortalece a relação e facilita a aceitação e implementação do feedback.

### Sugestão de Novos Assuntos

1. **Autoconhecimento e Liderança:** Como o autoconhecimento pode tornar líderes mais eficazes.
2. **Inteligência Emocional no Trabalho:** Desenvolvendo inteligência emocional para melhorar o ambiente corporativo.
3. **Gestão de Equipes Remotas:** Estrategias e ferramentas para gerenciar equipes que trabalham remotamente.
4. **Inovação no Ambiente Corporativo:** Como fomentar um ambiente de inovação dentro da empresa.
5. **Cultura Organizacional Positiva:** A importância de construir e manter uma cultura organizacional positiva.
6. **Diversidade e Inclusão:** Benefícios e práticas de diversidade e inclusão no ambiente de trabalho.
7. **Saúde Mental dos Colaboradores:** Promovendo o bem-estar mental no ambiente de trabalho.
8. **Liderança Feminina:** Desafios e triunfos das mulheres em posições de liderança.
9. **Sustentabilidade Corporativa:** Iniciativas sustentáveis e responsabilidade social nas empresas.
10. **Planejamento de Carreira:** Passos estratégicos para desenvolver uma carreira bem-sucedida e alinhada com seus valores e habilidades.

In [ ]:
biografias_df = pd.read_csv('/Users/elorr/Documents/backstage/dados/biografias.csv')
linkedin_posts_df = pd.read_csv('/Users/elorr/Documents/backstage/dados/noticias.csv')
artigos_df = pd.read_csv('/Users/elorr/Documents/backstage/dados/posts_linkedin.csv')

# i choose to remain with punctuation to keep the nuances of the text, given that i want to identify the "Ricardo way of speaking/writing"

#1. combine datasets into a single df
#2. remove duplicates combined_df.drop_duplicates(subset=['conteudo'], inplace=True)
#3. standardize formatting (convert everything to lower)
#4. tokenize the text
# clean 
# reconstruct 